In [ ]:
import pandas as pd

base = pd.read_csv('150_gold.csv')
finetuned_t5 = pd.read_csv('150_gold_t5.csv')
t5_base = pd.read_csv('t5_base_inference.csv')
extractive = pd.read_csv('extractive_models.csv')
ensemble = pd.read_csv('ensemble_models.csv')

In [ ]:
base = base[['episode_id', 'episode_name', 'transcript', 'episode_description', 'transcript', 'textrank_summary', 'lexrank_summary', 'lsa_summary', 'semi_supervised' ,'supervised', 'first_five']]
base.head(1)

,episode_id,episode_name,transcript,episode_description,transcript,textrank_summary,lexrank_summary,lsa_summary,semi_supervised,supervised,first_five
0,spotify:episode:08hXUWN6aOnHULXrqMiwTi,Recruiting Secrets From a MLM Recruiting Monst...,Hello everybody. What's going on in this Jess...,If you want to start mastering recruiting whic...,Hello everybody. What's going on in this Jess...,Let's download these stories about whoever I d...,You can do whatever networking events and I'm ...,We're going to talk about some powerful recrui...,"You cannot scummy hashtag wisely, or the peop...",In this episode I talk about some powerful re...,Hello everybody. What's going on in this Jess...


In [ ]:
finetuned_t5 = finetuned_t5[['episode_id', 't5_abstractive']]
finetuned_t5.head(1)

,episode_id,t5_abstractive
0,spotify:episode:08hXUWN6aOnHULXrqMiwTi,This is a great episode to start your network ...


In [ ]:
t5_base = t5_base[['episode_id', 't5_base']]
t5_base.head(1)

,episode_id,t5_base
0,spotify:episode:08hXUWN6aOnHULXrqMiwTi,If you want to be a network marketing recruiti...


In [ ]:
extractive = extractive[['episode_id', 'spanbert', 'bert', 'bert_coref', 'spanbert_base']]
extractive.head(1)

,episode_id,spanbert,bert,bert_coref,spanbert_base
0,spotify:episode:08hXUWN6aOnHULXrqMiwTi,"You cannot scummy hashtag wisely, or the peopl...","You cannot scummy hashtag wisely, or the peopl...","You cannot scummy hashtag wisely, or the peopl...","You cannot scummy hashtag wisely, or the peopl..."


In [ ]:
ensemble = ensemble[['episode_id', 'spanBert_t5', 'firstN_t5']]
ensemble.head(1)

,episode_id,spanBert_t5,firstN_t5
0,spotify:episode:08hXUWN6aOnHULXrqMiwTi,This is the #1 network marketing podcast in th...,This is a great podcast to start your network ...


In [ ]:
summaries = base.merge(finetuned_t5, left_on='episode_id', right_on='episode_id')
summaries = summaries.merge(t5_base, left_on='episode_id', right_on='episode_id')
summaries = summaries.merge(extractive, left_on='episode_id', right_on='episode_id')
summaries = summaries.merge(ensemble, left_on='episode_id', right_on='episode_id')
summaries.head(1)

,episode_id,episode_name,transcript,episode_description,transcript,textrank_summary,lexrank_summary,lsa_summary,semi_supervised,supervised,first_five,t5_abstractive,t5_base,spanbert,bert,bert_coref,spanbert_base,spanBert_t5,firstN_t5
0,spotify:episode:08hXUWN6aOnHULXrqMiwTi,Recruiting Secrets From a MLM Recruiting Monst...,Hello everybody. What's going on in this Jess...,If you want to start mastering recruiting whic...,Hello everybody. What's going on in this Jess...,Let's download these stories about whoever I d...,You can do whatever networking events and I'm ...,We're going to talk about some powerful recrui...,"You cannot scummy hashtag wisely, or the peop...",In this episode I talk about some powerful re...,Hello everybody. What's going on in this Jess...,This is a great episode to start your network ...,If you want to be a network marketing recruiti...,"You cannot scummy hashtag wisely, or the peopl...","You cannot scummy hashtag wisely, or the peopl...","You cannot scummy hashtag wisely, or the peopl...","You cannot scummy hashtag wisely, or the peopl...",This is the #1 network marketing podcast in th...,This is a great podcast to start your network ...


In [ ]:
summaries.rename(columns={
    'episode_id': 'Episode ID',
    'episode_name': 'Episode Name',
    'episode_description': 'Creator Description',
    'textrank_summary': 'TextRank',
    'lexrank_summary': 'LexRank',
    'lsa_summary': 'LSA',
    'semi_supervised': 'BART Base',
    'supervised': 'Fine-Tuned BART',
    'first_five': 'First Five Sentences',
    't5_abstractive': 'Fine-Tuned T5',
    't5_base': 'T5 Base',
    'spanbert': 'Coref Resolution + SpanBERT + K-Means',
    'bert': 'BERT + K-Means',
    'bert_coref': 'Coref Resolution + BERT + K-Means',
    'spanbert_base': 'SpanBERT + K-Means',
    'spanBert_t5': 'SpanBERT + T5',
    'firstN_t5': 'First 15 + T5'
}, inplace=True)

summaries.head(2)

,Episode ID,Episode Name,transcript,Creator Description,transcript,TextRank,LexRank,LSA,BART Base,Fine-Tuned BART,First Five Sentences,Fine-Tuned T5,T5 Base,Coref Resolution + SpanBERT + K-Means,BERT + K-Means,Coref Resolution + BERT + K-Means,SpanBERT + K-Means,SpanBERT + T5,First 15 + T5
0,spotify:episode:08hXUWN6aOnHULXrqMiwTi,Recruiting Secrets From a MLM Recruiting Monst...,Hello everybody. What's going on in this Jess...,If you want to start mastering recruiting whic...,Hello everybody. What's going on in this Jess...,Let's download these stories about whoever I d...,You can do whatever networking events and I'm ...,We're going to talk about some powerful recrui...,"You cannot scummy hashtag wisely, or the peop...",In this episode I talk about some powerful re...,Hello everybody. What's going on in this Jess...,This is a great episode to start your network ...,If you want to be a network marketing recruiti...,"You cannot scummy hashtag wisely, or the peopl...","You cannot scummy hashtag wisely, or the peopl...","You cannot scummy hashtag wisely, or the peopl...","You cannot scummy hashtag wisely, or the peopl...",This is the #1 network marketing podcast in th...,This is a great podcast to start your network ...
1,spotify:episode:0CExTNH4LFqp1ec1mhTd4I,A Public Service Announcement from the mrbrown...,"Ladies, have you been molested don't be a vic...",Don't be a silent victim of crime. a parody fr...,"Ladies, have you been molested don't be a vic...",Have someone escort you home when it's late ab...,But you can touch his life. But you can touch ...,If you only touched you in minor ways never mi...,If you have been molested call the police sho...,"If you have been molested, here are some of t...","Ladies, have you been molested don't be a vic...",This is a Public Service Announcement for Mr. ...,"Ladies, have you been molested don't be a vict...","Ladies, have you been molested don't be a vict...","Ladies, have you been molested don't be a vict...","Ladies, have you been molested don't be a vict...","Ladies, have you been molested don't be a vict...",--- Send in a voice message: https://anchor.fm...,This is a Public Service Announcement for Mr. ...


In [ ]:
del summaries['transcript']
summaries.tail(2)

,Episode ID,Episode Name,Creator Description,TextRank,LexRank,LSA,BART Base,Fine-Tuned BART,First Five Sentences,Fine-Tuned T5,T5 Base,Coref Resolution + SpanBERT + K-Means,BERT + K-Means,Coref Resolution + BERT + K-Means,SpanBERT + K-Means,SpanBERT + T5,First 15 + T5
148,spotify:episode:7uv6JrgV01fiecvCgxRMy8,Best Organic Dessert Recipes,Comfort Foods for Cold Nights,Kosher salt 3 cups all-purpose flour plus more...,Four cups whole milk warmed pinch of freshly g...,Finely chopped kosher salt and freshly ground ...,"Ingredients bolognese sauce 1 large onion, 1 ...",1 cup dry white wine 1 cup whole milk 1 14.5 ...,Ingredients bolognese sauce 1 large onion coa...,This episode is about bolognese sauce. It's al...,Ingredients bolognese sauce 1 large onion coar...,Ingredients bolognese sauce 1 large onion coar...,Ingredients bolognese sauce 1 large onion coar...,Ingredients bolognese sauce 1 large onion coar...,Ingredients bolognese sauce 1 large onion coar...,"This episode is about Bolognese Sauce, a sauce...","This episode is about Bolognese Sauce, a sauce..."
149,spotify:episode:7zd15Qptaa1rxz4QCbHotX,BREAKING EMERGENCY POD: TYLER BARISHAW LIKELY ...,BREAKING EMERGENCY POD: TYLER BARISHAW LIKELY ...,I just hope you enjoy it and hope you may make...,"Yeah, anyways, you know, you get like free cho...",This is just it's I don't even know what NBA ...,Sean Kershaw is joining the Marines . He will...,In the midst of signing a contract with the M...,Welcome to the good life. Welcome to the good...,This is a very interesting episode. We talk ab...,A quarter of military trade in his head will b...,He will be doing it you will be doing it a qua...,He will be doing it you will be doing it a qua...,He will be doing it you will be doing it a qua...,He will be doing it you will be doing it a qua...,This week we talk about the importance of bein...,"This is not a joke, but it's a great opportuni..."


In [ ]:
import string

def clean_up(text):
  head, _ , _ = text.partition(' ---')
  head = head.strip()
  first_letter = head[0].capitalize()
  head = first_letter + head[1:]
  if head[-1] in string.punctuation:
    pass
  else:
    head += '.'
  return head

def clean_up_base(text):
  head, _ , _ = text.partition(' ---')
  head = head.strip()
  return head


summaries['Fine-Tuned BART'] = summaries.apply(lambda row: clean_up(row['Fine-Tuned BART']), axis = 1)
#summaries['Creator Description'] = summaries.apply(lambda row: clean_up_base(row['Creator Description']), axis = 1)


In [ ]:
#summaries.to_csv('summaries.csv')

#Rouge Scores

In [ ]:
!pip install rouge
from rouge import Rouge
rouge = Rouge()

In [ ]:
summaries.columns.values

array(['Episode ID', 'Episode Name', 'Creator Description', 'TextRank',
       'LexRank', 'LSA', 'BART Base', 'Fine-Tuned BART',
       'First Five Sentences', 'Fine-Tuned T5', 'T5 Base',
       'Coref Resolution + SpanBERT + K-Means', 'BERT + K-Means',
       'Coref Resolution + BERT + K-Means', 'SpanBERT + K-Means',
       'SpanBERT + T5', 'First 15 + T5'], dtype=object)

In [ ]:
#summaries['Creator Description'] = summaries.apply(lambda row: row['Creator Description'] if row['Creator Description'] != '' else row['Episode Name'], axis = 1)

In [ ]:
row_names = ['TextRank', 'LexRank', 'LSA', 'BART Base', 'Fine-Tuned BART', 'First Five Sentences', 'Fine-Tuned T5', 'T5 Base', 'Coref Resolution + SpanBERT + K-Means', 'BERT + K-Means', 'Coref Resolution + BERT + K-Means', 'SpanBERT + K-Means', 'SpanBERT + T5', 'First 15 + T5']

textrank = rouge.get_scores(summaries['TextRank'], summaries['Creator Description'],  avg=True)
lexrank = rouge.get_scores(summaries['LexRank'], summaries['Creator Description'], avg=True)
lsa = rouge.get_scores(summaries['LSA'], summaries['Creator Description'], avg=True)
bart_base = rouge.get_scores(summaries['BART Base'], summaries['Creator Description'], avg=True)
bart_finetuned = rouge.get_scores(summaries['Fine-Tuned BART'], summaries['Creator Description'], avg=True)
first_five = rouge.get_scores(summaries['First Five Sentences'], summaries['Creator Description'], avg=True)
t5_finetuned = rouge.get_scores(summaries['Fine-Tuned T5'], summaries['Creator Description'], avg=True)
t5_base = rouge.get_scores(summaries['T5 Base'], summaries['Creator Description'], avg=True)
coref_spanBert = rouge.get_scores(summaries['Coref Resolution + SpanBERT + K-Means'], summaries['Creator Description'], avg=True)
bert_kmeans = rouge.get_scores(summaries['BERT + K-Means'], summaries['Creator Description'], avg=True)
coref_bert = rouge.get_scores(summaries['Coref Resolution + BERT + K-Means'], summaries['Creator Description'], avg=True)
spanBert_kmeans = rouge.get_scores(summaries['SpanBERT + K-Means'], summaries['Creator Description'], avg=True)
spanBert_t5 = rouge.get_scores(summaries['SpanBERT + T5'], summaries['Creator Description'], avg=True)
first15_t5 = rouge.get_scores(summaries['First 15 + T5'], summaries['Creator Description'], avg=True)

In [ ]:
rouge1_scores = [textrank['rouge-1']['f'], lexrank['rouge-1']['f'], lsa['rouge-1']['f'], bart_base['rouge-1']['f'], bart_finetuned['rouge-1']['f'], first_five['rouge-1']['f'], t5_finetuned['rouge-1']['f'], t5_base['rouge-1']['f'], coref_spanBert['rouge-1']['f'], bert_kmeans['rouge-1']['f'], coref_bert['rouge-1']['f'], spanBert_kmeans['rouge-1']['f'], spanBert_t5['rouge-1']['f'], first15_t5['rouge-1']['f']]
rouge1_df = pd.DataFrame({'Rouge1-F':rouge1_scores}, index=row_names)

In [ ]:
rouge2_scores = [textrank['rouge-2']['f'], lexrank['rouge-2']['f'], lsa['rouge-2']['f'], bart_base['rouge-2']['f'], bart_finetuned['rouge-2']['f'], first_five['rouge-2']['f'], t5_finetuned['rouge-2']['f'], t5_base['rouge-2']['f'], coref_spanBert['rouge-2']['f'], bert_kmeans['rouge-2']['f'], coref_bert['rouge-2']['f'], spanBert_kmeans['rouge-2']['f'], spanBert_t5['rouge-2']['f'], first15_t5['rouge-2']['f']]
rouge2_df = pd.DataFrame({'Rouge2-F':rouge2_scores}, index=row_names)

In [ ]:
rougel_scores = [textrank['rouge-l']['f'], lexrank['rouge-l']['f'], lsa['rouge-l']['f'], bart_base['rouge-l']['f'], bart_finetuned['rouge-l']['f'], first_five['rouge-l']['f'], t5_finetuned['rouge-l']['f'], t5_base['rouge-l']['f'], coref_spanBert['rouge-l']['f'], bert_kmeans['rouge-l']['f'], coref_bert['rouge-l']['f'], spanBert_kmeans['rouge-l']['f'], spanBert_t5['rouge-l']['f'], first15_t5['rouge-l']['f']]
rougel_df = pd.DataFrame({'RougeL-F':rougel_scores}, index=row_names)

In [ ]:
rouge_f_scores = rouge1_df.merge(rouge2_df, left_index=True, right_index = True)
rouge_f_scores = rouge_f_scores.merge(rougel_df, left_index=True, right_index = True)
rouge_f_scores

,Rouge1-F,Rouge2-F,RougeL-F
TextRank,0.116462,0.014054,0.104729
LexRank,0.119708,0.014022,0.106071
LSA,0.132847,0.010752,0.109076
BART Base,0.161447,0.029776,0.146231
Fine-Tuned BART,0.164612,0.039461,0.147935
First Five Sentences,0.138515,0.029270,0.126388
Fine-Tuned T5,0.172924,0.041895,0.159610
T5 Base,0.147173,0.023238,0.131217
Coref Resolution + SpanBERT + K-Means,0.145516,0.026931,0.124303
BERT + K-Means,0.137192,0.020735,0.116341


In [ ]:
rouge1_scores = [textrank['rouge-1']['p'], lexrank['rouge-1']['p'], lsa['rouge-1']['p'], bart_base['rouge-1']['p'], bart_finetuned['rouge-1']['p'], first_five['rouge-1']['p'], t5_finetuned['rouge-1']['p'], t5_base['rouge-1']['p'], coref_spanBert['rouge-1']['p'], bert_kmeans['rouge-1']['p'], coref_bert['rouge-1']['p'], spanBert_kmeans['rouge-1']['p'], spanBert_t5['rouge-1']['p'], first15_t5['rouge-1']['p']]
rouge1_df = pd.DataFrame({'Rouge1-F':rouge1_scores}, index=row_names)

rouge2_scores = [textrank['rouge-2']['p'], lexrank['rouge-2']['p'], lsa['rouge-2']['p'], bart_base['rouge-2']['p'], bart_finetuned['rouge-2']['p'], first_five['rouge-2']['p'], t5_finetuned['rouge-2']['p'], t5_base['rouge-2']['p'], coref_spanBert['rouge-2']['p'], bert_kmeans['rouge-2']['p'], coref_bert['rouge-2']['p'], spanBert_kmeans['rouge-2']['p'], spanBert_t5['rouge-2']['p'], first15_t5['rouge-2']['p']]
rouge2_df = pd.DataFrame({'Rouge2-F':rouge2_scores}, index=row_names)

rougel_scores = [textrank['rouge-l']['p'], lexrank['rouge-l']['p'], lsa['rouge-l']['p'], bart_base['rouge-l']['p'], bart_finetuned['rouge-l']['p'], first_five['rouge-l']['p'], t5_finetuned['rouge-l']['p'], t5_base['rouge-l']['p'], coref_spanBert['rouge-l']['p'], bert_kmeans['rouge-l']['p'], coref_bert['rouge-l']['p'], spanBert_kmeans['rouge-l']['p'], spanBert_t5['rouge-l']['p'], first15_t5['rouge-l']['p']]
rougel_df = pd.DataFrame({'RougeL-F':rougel_scores}, index=row_names)

rouge_p_scores = rouge1_df.merge(rouge2_df, left_index=True, right_index = True)
rouge_p_scores = rouge_p_scores.merge(rougel_df, left_index=True, right_index = True)
rouge_p_scores

,Rouge1-F,Rouge2-F,RougeL-F
TextRank,0.095777,0.011637,0.091729
LexRank,0.128981,0.014028,0.117497
LSA,0.116140,0.009472,0.096376
BART Base,0.168322,0.030159,0.153403
Fine-Tuned BART,0.244991,0.060978,0.214995
First Five Sentences,0.140695,0.028494,0.130149
Fine-Tuned T5,0.249061,0.062894,0.222959
T5 Base,0.179698,0.026745,0.155676
Coref Resolution + SpanBERT + K-Means,0.154910,0.031440,0.133606
BERT + K-Means,0.122645,0.018572,0.106751


In [ ]:
rouge1_scores = [textrank['rouge-1']['r'], lexrank['rouge-1']['r'], lsa['rouge-1']['r'], bart_base['rouge-1']['r'], bart_finetuned['rouge-1']['r'], first_five['rouge-1']['r'], t5_finetuned['rouge-1']['r'], t5_base['rouge-1']['r'], coref_spanBert['rouge-1']['r'], bert_kmeans['rouge-1']['r'], coref_bert['rouge-1']['r'], spanBert_kmeans['rouge-1']['r'], spanBert_t5['rouge-1']['r'], first15_t5['rouge-1']['r']]
rouge1_df = pd.DataFrame({'Rouge1-F':rouge1_scores}, index=row_names)

rouge2_scores = [textrank['rouge-2']['r'], lexrank['rouge-2']['r'], lsa['rouge-2']['r'], bart_base['rouge-2']['r'], bart_finetuned['rouge-2']['r'], first_five['rouge-2']['r'], t5_finetuned['rouge-2']['r'], t5_base['rouge-2']['r'], coref_spanBert['rouge-2']['r'], bert_kmeans['rouge-2']['r'], coref_bert['rouge-2']['r'], spanBert_kmeans['rouge-2']['r'], spanBert_t5['rouge-2']['r'], first15_t5['rouge-2']['r']]
rouge2_df = pd.DataFrame({'Rouge2-F':rouge2_scores}, index=row_names)

rougel_scores = [textrank['rouge-l']['r'], lexrank['rouge-l']['r'], lsa['rouge-l']['r'], bart_base['rouge-l']['r'], bart_finetuned['rouge-l']['r'], first_five['rouge-l']['r'], t5_finetuned['rouge-l']['r'], t5_base['rouge-l']['r'], coref_spanBert['rouge-l']['r'], bert_kmeans['rouge-l']['r'], coref_bert['rouge-l']['r'], spanBert_kmeans['rouge-l']['r'], spanBert_t5['rouge-l']['r'], first15_t5['rouge-l']['r']]
rougel_df = pd.DataFrame({'RougeL-F':rougel_scores}, index=row_names)

rouge_r_scores = rouge1_df.merge(rouge2_df, left_index=True, right_index = True)
rouge_r_scores = rouge_r_scores.merge(rougel_df, left_index=True, right_index = True)
rouge_r_scores

,Rouge1-F,Rouge2-F,RougeL-F
TextRank,0.226912,0.029748,0.173667
LexRank,0.173084,0.023050,0.139604
LSA,0.207760,0.015353,0.163668
BART Base,0.209020,0.040502,0.182396
Fine-Tuned BART,0.168525,0.039769,0.147657
First Five Sentences,0.200325,0.049853,0.172371
Fine-Tuned T5,0.188136,0.043149,0.168546
T5 Base,0.161536,0.028513,0.141191
Coref Resolution + SpanBERT + K-Means,0.181187,0.032933,0.147303
BERT + K-Means,0.208740,0.034314,0.165000
